# Hito 3: Corona virus (Covid-19)

## Presentación

Somos el equipo conformado por Raúl Cid, José Espina, Michelle Valenzuela y Alejandro Veragua. Nuestro *dataset* corresponde al [publicado en el portal Kaggle](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset) construido y mantenido por la Universidad John Hopkins, al igual que para el hito 1

## Comentarios sobre el hito 1

Se realizó un análisis exploratorio, donde descubrimos varios datos inconsistentes y outliers
Se buscaron correlaciones contra atributos como el ingreso per cápita, la temperatura, el índice de pobreza de los diferentes países, pero sin éxito. Finalmente se realizó una prueba de concepto de clústering a partir de los coeficientes de un modelo de regeresión polinomial donde sí aparecieron resultados interesantes

## Propuesta del hito 3
Considerando que los polinomios hacen "overfitting" cuando el grado es muy alto, como el ejercicio realizado en el hito 1, aprovecharemos hacer algo similar, con la diferencia de usar un modelo auto-regresivo y hacer clustering sobre los coeficientes que resulten del modelo. En particular, quisimos experimentar con un modelo auto-regresivo con media móvil (ARMA). Como se tienen datos sólo de los últimos meses y el virus es nuevo, los datos no son estacionarios (o no hay suficientes datos para demostrar que lo son), por lo que usaremos una variante llamada ARIMA (donde la "i" hace referencia a la integración), que permite encontrar las diferencias no-estacionarias para lograr la estacionariedad (https://www.statisticshowto.com/arma-model/)

Además, se refuerza la idea de usar el modelo ARIMA ya que lo usan en 2 artículos publicados en la literatura:
1) Benvenuto, D., Giovanetti, M., Vassallo, L., Angeletti, S., & Ciccozzi, M. (2020). Application of the ARIMA model on the COVID-2019 epidemic dataset. Data in brief, 105340.
2) Yang, Q., Wang, J., Ma, H., & Wang, X. (2020). Research on COVID-19 Based on ARIMA ModelΔ—Taking Hubei, China as an example to see the epidemic in Italy. Journal of Infection and Public Health.

## Metodología de trabajo para el hito 3
Los siguientes pasos describen, a grandes rasgos, nuestra metodología planificada:
* Preprocesamiento de los datos, con el objetivo de remover los datos outliers. Descubrimos en el hito 1 que hay paises con datos demasiado alejados de la tendencia central, retroceso en datos acumulados (que debiesen ser estrictamente crecientes) y datos con ruido, como los del crucero Diamond Princess
* Cálculo de coeficientes de modelo predictivo ARIMA, y su predicción para Chile
* Clustering de los coeficientes

###NOTA IMPORTANTE 1
Para ejecutar este notebook, se necesitan scikit, matplotlib, numpy y statsmodels. Éste último para poder aplicar el modelo ARIMA. Se debe instalar con pip en el ambiente del libro:

$ pip install statsmodels

o con conda:

$ conda install -c conda-forge statsmodels

(https://www.statsmodels.org/stable/install.html)

###NOTA IMPORTANTE 2
Calcular el modelo ARIMA toma un tiempo de cómputo importante

## Configuración base del notebook
El bloque a continuación configura el libro, definiendo funciones y cargando las fuentes de datos

In [ ]:
# Librería de álgebra lineal
import numpy as np 
# Procesamiento de datos y carga de archivos CSV
import pandas as pd
# Librearía gráfica
import matplotlib.pylab as plt
# Librería para usar recursos del sistema operativo
import os 
# Librearía usada para extraer el nombre del archivo de cada path
import ntpath
# Librería gráfica
import seaborn as sns
# Librearía para usar herramientas relacionadas con
# aprendizaje máquina. Se usó para preprocesar y
# calcular regresión
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# Colores
from matplotlib import colors
# Ticker
import matplotlib.ticker as ticker
# Animation
import matplotlib.animation as animation
# Arima
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
# Configuración general
%load_ext autoreload
%autoreload 2

# Matplotlib inline
%matplotlib notebook
%matplotlib inline

# Se prepara carga de los archivos del dataset
paths = []
for dirname, _, filenames in os.walk('./novel-corona-virus-2019-dataset'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        paths.append(path);
dataFrames = {}
for path in paths:
    dataFrames[ntpath.basename(path)] = pd.read_csv(path)

## Carga y reduce de datos por dia

Se agrupan los atributos por indexados por fecha y país, para una manipulación más simple de los datos

In [ ]:
from atributos_extra.correccion_nombres import cargar_correccion, atributos_extra
covid19Data = dataFrames['covid_19_data.csv'].copy()
dict_error, dict_pares = cargar_correccion()
group_covid_data, paises_covid_df = atributos_extra(covid19Data, dict_error)
group_covid_data.columns = group_covid_data.columns.droplevel(1)
group_covid_data['ObservationDate'] = pd.to_datetime(group_covid_data['ObservationDate'])
group_covid_data.set_index('Country/Region', inplace=True)
group_covid_data.set_index('ObservationDate', inplace=True, append=True)
group_covid_data.drop('Last Update', axis=1, inplace=True)
# se limpia Laos por valores NaN
group_covid_data.drop('Laos', level=0, inplace=True)

In [ ]:
paises_covid_df.head(3)

In [ ]:
group_covid_data.loc['Chile']

## Calcular fenomeno por dia, basado en acumulados

En el dataset vienen los acumulados de los contagios, fallecidos y recuperados. Se implementa un pequeño algoritmo que permite calcular los esos mismos atributos para cada día. Además, determinamos la fecha del primer contagio, de tal manera de tener una columna con el número de día desde el primer contagio, con el fin de poder comparar los países entre sí

In [ ]:
import datetime
def apply_primer_dia(data):
    date_inicio_dataset = pd.to_datetime('22/01/2020') #datetime.date(2020,1,22)
    #data = data.sort_index(ascending=True, level=0)
    observation_date = data.index.get_level_values(1)
    
    num_dia_desde_primer_caso = 0
    data[ 'num_dia_desde_22_enero'] = observation_date - date_inicio_dataset
    
    fecha_primer_caso = observation_date.min()
    data['num_dia_desde_primer_caso'] = observation_date - fecha_primer_caso
    # Set indexed
    data.set_index('num_dia_desde_primer_caso', inplace=True, append=True)
    # data.set_index('num_dia_desde_22_enero', inplace=True, append=True)
    data.reset_index(level=1, inplace=True)
    return data

def compute_diff(col):
    diff = col - col.shift(periods=1)
    diff[0] = col[0]
    # print(col[10])
    # print(diff[9])
    # print(20*'-')
    return diff

def apply_por_dia(data):
    data['num_confirmados'] = compute_diff(data['Confirmed'])
    data['num_recuperados'] = compute_diff(data['Recovered'])
    data['num_fallecidos'] = compute_diff(data['Deaths'])
    return data

data = (
    group_covid_data
        .groupby('Country/Region')
        .apply(lambda country: apply_primer_dia(country))
        .droplevel(1)
        .groupby(level=0)
        .apply(lambda country: apply_por_dia(country))
)

data.loc['Chile'].sort_index(ascending=False).head(5)

## ¿Como se comporta Chile en cantidad de confirmados?
Se puede apreciar el "pic" en el momento en que el MISNAL corrigió los datos

Además, se puede ver que había una tendencia a la baja que se rompe en los últimos días

In [ ]:
data.loc['Chile']['num_confirmados'].plot()

### Preproceso: Outliers
Como parte del preprocesamiento, se realiza el siguiente tratamiento a los datos: Para cada país, se revisan las columnas de confirmados, fallecidos y recuperados. En cada una de ellas se buscan outliers, en este caso, valores que superen en magnitud al percentil 99 o que sean menores a cero. Estos se transorman a NaN y luego interpolamos los datos para llenar estos espacios.
Además, se debe recalcular la cantidad acumulada por día para cada columna.

In [ ]:
def remove_outliers(dset_base, cols=['num_confirmados', 'num_fallecidos', 'num_recuperados']):  
    for pais in dset_base['Country/Region'].unique():
        for col in cols:
            pais_col = dset_base[dset_base['Country/Region']==str(pais)][col]
            changed_values = pd.Series(np.nan, index=pais_col.index)
            for value in pais_col.iteritems():
                if value[1] > pais_col.quantile(.99) or value[1] < 0:
                    changed_values[value[0]] = np.nan
                else:
                    changed_values[value[0]] = value[1]
            changed_values.interpolate(inplace=True)
            acc = changed_values.cumsum()
            dset_base.loc[(dset_base['Country/Region']==pais), col] = changed_values
            dset_base.loc[(dset_base['Country/Region']==pais), str('acc') + col[3:]] = acc
    return dset_base

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

def remove_outliers(col, q=0.99):
    threshold = col.mean() + 3*col.std()
    # threshold = col.quantile(q)
    col[(col >= threshold)  | (col < 0)] = np.nan
    # col[(col >= col.quantile(q)) | (col < 0)] = np.nan
    #print(col.name, col.isna().sum())
    res = col.interpolate()
    return res

def apply_remove_outliers(pais_col, cols=['num_confirmados', 'num_recuperados']):
    # print(pais_col.name)
    pais_col[cols] = pais_col[cols].apply(lambda x: remove_outliers(x))
    # print(pais_col[cols].isna().sum())
    return pais_col

def compute_acumulados(pais_col):
    pais_col['Confirmed'] = pais_col['num_confirmados'].cumsum()
    pais_col['Recovered'] = pais_col['num_recuperados'].cumsum()
    pais_col['Deaths'] = pais_col['num_fallecidos'].cumsum()
    pais_col = pais_col.rename(
        columns={
            'Confirmed': 'acc_confirmados',
            'Recovered': 'acc_recuperados',
            'Deaths': 'acc_fallecidos'
        }
    )
    return pais_col
# data = data.drop('Papua New Guinea')
res = (
    data
        .groupby(level=0)
        .apply(lambda pais_col: apply_remove_outliers(pais_col))
    )

# Calculamos los acumulados
res = (res
    .groupby(level=0)
    .apply(lambda pais_col: compute_acumulados(pais_col))
    )

### Datos limpios

Ahora contamos con los datos limpios de anomalías estadísticas, agrupados por país y fecha, aprovechando la capacidad de los índices del tipo de datos DataFrame del módulo Pandas

In [ ]:
res.loc['Chile'].sort_index(level=0, ascending=False).head(5)

### ¿Como está Chile, en cantidad de confirmados, recuperados y fallecidos?

Ahora, con los datos limpios, es posible ver la progresión de contagios, recuperados y fallecidos en Chile

Por el momento, se podría pensar que lo pero ya pasó, sin embargo, en Europa los países han tenido rebrotes, como es el caso de Japón (ver figura a continuación a la de Chile)

In [ ]:
figsize = (14, 5)
res.loc['Chile'][['num_confirmados', 'num_recuperados', 'num_fallecidos']].plot(figsize=figsize, grid=True, title='Fenomenos por día')
res.loc['Chile'][['acc_confirmados', 'acc_recuperados', 'acc_fallecidos']].plot(figsize=figsize, grid=True, title='Fenomenos por dia, acumulado')


In [ ]:
# Caso de rebrote en Japón, en el día 180, aproximadamente, desde el primer contagio
figsize = (14, 5)
res.loc['Japan'][['num_confirmados', 'num_recuperados', 'num_fallecidos']].plot(figsize=figsize, grid=True)
res.loc['Japan'][['acc_confirmados', 'acc_recuperados', 'acc_fallecidos']].plot(figsize=figsize, grid=True)

## "Polinomial features"

Se procede a repetir el experimento del hito 1 con los datos actualizados. Nos gustaría demostrar que agrupar (cluster) mediante los coeficientes de ARIMA es mejor idea que con los polinomios, que fue lo que hicimos la primera oportunidad

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import pickle

def load_model(pais):
    filename_predictor = './models/polynomial_predictor_{}.joblib'.format(pais)
    filename_features = './models/polynomial_features_{}.joblib'.format(pais)
    print('Loading model {}'.format(filename_predictor))
    print('Loading model {}'.format(filename_features))
    return load_pickle(filename_predictor), load_pickle(filename_features)
    

def to_Xy(dset_pais, over='num_confirmados'):
    X = np.linspace((0,),(len(dset_pais.index)-1,),len(dset_pais.index))
    y = dset_pais[over].values
    return X, y
    
def save_pickle(filename, model):
    with open(filename,'wb') as outfile: 
        pickle.dump(model, outfile)
        
def load_pickle(filename):
    with open(filename,'rb') as outfile:
        return pickle.load(outfile)

def apply_features(dset_pais, degree=6,save=False, pais_tosave=['Chile', 'USA', 'Japan'], over='num_confirmados'):
    pais = dset_pais.index.get_level_values(0)[0]
    X = np.linspace((0,),(len(dset_pais.index)-1,),len(dset_pais.index))
    y = dset_pais[over].values

    
    poly_reg = PolynomialFeatures(degree=degree)
    X_poly = poly_reg.fit_transform(X)
    
    pol_reg = LinearRegression()
    pol_reg.fit(X_poly, y)

    if save and pais in pais_tosave:
        filename_predictor = './models/polynomial_predictor_{}.joblib'.format(pais)
        filename_features = './models/polynomial_features_{}.joblib'.format(pais)
        save_pickle(filename_predictor, pol_reg)
        save_pickle(filename_features, poly_reg)

    return pd.Series(poly_reg.transform(X)[-1])
    
def viz_polymonial(pol_reg, poly_reg, X):
    plt.figure(figsize=(7,7))
    plt.scatter(X, y, color='red')
    plt.plot(X, pol_reg.predict(poly_reg.fit_transform(X)), color='blue')
    plt.title('Regresión polinomial de los contagiados de Chile')
    plt.xlabel('dias desde el contagio')
    plt.ylabel('Contagiados acumulados')
    plt.savefig('figures/chile_polinomial_reg.png')
    plt.grid(True)
    return


In [ ]:
# Calcular features
from src.clustering import TSNEVisualizer, ClusteringTunning
over = 'num_confirmados'
features = (
    res
        .groupby(level=0)
        .apply(lambda dset_pais: apply_features(dset_pais, degree=7, save=True, over=over))
)
features.drop(0, axis=1, inplace=True)
pais = 'Chile'
chile_predictor, chile_features = load_model(pais)
X, y = to_Xy(res.loc[pais], over=over)
viz_polymonial(chile_predictor, chile_features, X)

## Agrupamiento

In [ ]:
from src.clustering import TSNEVisualizer, ClusteringTunning

In [ ]:
from sklearn.preprocessing import StandardScaler
raw_standard = StandardScaler().fit_transform(features.values)
standard_feats = pd.DataFrame(raw_standard, columns = features.columns, index=features.index)
standard_feats

In [ ]:
# Entrenar el GMM
visualizer = TSNEVisualizer()
clustering = ClusteringTunning(visualizer).fit(standard_feats, n_components=2)
prediction = clustering.predict(standard_feats)
# clustering.scatter()
clustering.estimator_

# Plotear resultado del clustering
clustering.plot_clustering(name='GMM')

In [ ]:
clustering.scatter()

## Cálculo del análisis de silueta

El análsis de silueta nospermite demostrar 

In [ ]:
from sklearn.metrics import silhouette_score
print(silhouette_score(features, prediction.values.ravel()))

## Modelo ARIMA
Usamos un modelo ARIMA p=5,d=1,q=0. El detalle de estos parámetros se explica a continuación (basado en https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/)

* El parámetro "p" es el número de observaciones pasadas que considera el modelo. También se le llama "orden".
* El parámetro "d" es el grado de diferenciación. Esto se usa para generar estacionariedad en series no estacionarias
* El parámetro "q" es el tamaño de la media móvil para el error de los datos anteriores

In [ ]:
# Se calculan coeficientes del ARIMA por país. Si ya está calculado se lee el csv
# Los parámetros usados se escogieron en base a la moda, al calcular el óptimo para cada país
dframe_arima = None
if os.path.isfile('arima_coefs.csv') :
    dframe_arima = pd.read_csv('arima_coefs.csv')
else :
    pais_arima_coef = dict()
    paises_error = []
    for pais in dset_base.pais.unique() :
        dbset_actual = dset_base[dset_base['pais']==pais][['num_dia_desde_primer_caso','num_confirmados']]
        dbset_actual= dbset_actual.reset_index()
        series_actual = pd.Series(dbset_actual.num_confirmados.values, index=dbset_actual.num_dia_desde_primer_caso)
        series_actual.index = pd.DatetimeIndex(series_actual.index).to_period('D')
        try :
            model = ARIMA(series_actual.astype(float), order=(5,1,0),missing="drop")
            model_fit = model.fit(disp=False)
            pais_arima_coef[pais] = model_fit.params
        except :
            paises_error.append(pais)
    dframe_arima = pd.DataFrame(data=pais_arima_coef)
    dframe_arima.to_csv('arima_coefs.csv')


### Coeficientes ARIMA

Se muestra un resumen de los coeficientes calculados del modelo ARIMA por país

In [ ]:
dframe_arima

## Predicción para Chile

En el anexo está el cómo encontrar los parámetros óptimos para Chile

In [ ]:
dbset_actual = res.loc['Chile']['num_confirmados']
dbset_actual= dbset_actual.reset_index()
series_actual = pd.Series(dbset_actual.num_confirmados.values, index=dbset_actual.num_dia_desde_primer_caso)
model = ARIMA(series_actual.astype(float), order=(6,0,0),missing="drop")
model_fit = model.fit(disp=False)
model_fit.plot_predict(2,160);

## Análisis de los resultados y discusión

- Es importante contar con datos suficientemente limpios (bien estructurados y poco ruido). No es menor la cantidad de tiempo invertido en la limpieza tanto para el hito 1 como este hito.
- El ruido de los datos se deriva principalmente del modo en que estos se han estado contabilizando por pais, por ejemplo: Para el caso de chile en la columan confirmados se presenta un peak que se eleva de 5000 a 35000 (aprox), este se debio a que el minsal no habia reportado una cantidad considerable de casos por considerarlos como 'pendientes'. La mayoria de los errores y ruidos en el dataset son de similar indole.
- Es importante eliminar o suavizar lo mas posible el ruido debido a que al realizar clustering se podria estar agrupando por 'tipo de ruido' y o por tipo de curva o evolucion. Se sigue una logica similar respecto a ruido y modelamiento.
- Se puede observar, en el gráfico de la predicción, que la cuarentena y el confinamiento han dado resultado y el país estaba bajando la tasa de contagio. Lamentablemente el gobierno está apresurando el regreso a "la normalidad", y podría suceder que haya un rebrote del virus
- Se puede observar en el gráfico, en la sección del clustering, que la idea de "clusterizar" los coeficientes del modelo predictivo ARIMA no funcionó, como sí había funcionado con los coeficientes del polinomio de regresión del hito 1
- Aún así, aprovechamos que tenemos el modelo ARIMA funcionando, y una búsqueda sus hiperparámetros para Chile (ver Anexo) para hacer una predicción para los próximos 20 días
- No se encuentra evidencia significativa para afirmar que en efecto existen 'tipos' de curva o paises, coeficientes provenientes del modelo no parecieran tener una naturaleza adecuada para realizar esta tarea. Por otra parte, durante la revision de los clusters no se observa ningun patron distintivo, paises de cluster a y b (+++) no presentan caracteristicas diferenciales.

Conclusiones:

- Si bien el clustering no resulto ser efectivo es posible sacar algunas conclusiones a partir de esto: o bien los datos no son suficientemente trabajables (puede que no tenga sentido realizar clustering sobre ellos) o bien, puede ser que verdaderamente solo existe un tipo de tendencia y cada pais solo describira esta y no otra curva. Un motivo de porque esto es factible es la gran ineficiencia con la cual han actuado los distintos gobiernos, de modo que el qe solo exista un tipo de curva de avance no es algo tan extraño
- El modelo pareciera ser suficientemente bueno para realizar predicciones a corto plazo (maximo 15-20)
- Las técnicas de minería de datos permiten analizar el comportamiento de los datos y ayudar en la toma de desiciones. El Covid-19 ha generado grandes cambios en la sociedad, ha fallecido mucha gente y nos ha hecho cuestionar nuestra manera de actuar ante infecciones y problemas sanitarios. Proponemos que la minería de datos debiese ser parte de la política como soporte clave y permanente en la toma de desiciones estratégicas en los gobiernos actuales


## Trabajo futuro

Nos hubiese gustado haber podido seguir experimentando en las siguientes líneas:
* Demostrar, empíricamente, si las medidas del gobierno, tanto el confinamiento y desconfinamiento, han sido apropiadas para dismunuir la tasa de contagios
* Modelos predictivos basados en ARIMA para todos los países del mundo, y no sólo Chile
* Aplicar ARIMA por intervalos de tiempo, donde podría mejorar la predicción

## Anexo

### Cálculo de hiper-parámetros del modelo ARIMA para Chile

Basado en https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/

In [ ]:
import warnings

# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
				except:
					continue
	return {'best_cfg' : best_cfg, 'best_score' : best_score}
 

# evaluate parameters
p_values = range(4, 11)
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
bests = dict()
dbset_actual = res.loc['Chile']['num_confirmados']
dbset_actual= dbset_actual.reset_index()
series_actual = pd.Series(dbset_actual.num_confirmados.values, index=dbset_actual.num_dia_desde_primer_caso)
print(evaluate_models(series_actual.values, p_values, d_values, q_values))

## Anexo II: Análisis de clusters

In [ ]:
prediction.rename(columns={0: 'cluster_num'}, inplace=True)
prediction

In [ ]:
paises_covid_df.describe()

In [ ]:
prediction

In [ ]:
paises_covid_df.rename(columns={'pais': 'Country/Region'}, inplace=True)

In [ ]:
paises_covid_df.set_index('Country/Region').merge(prediction, right_index=True, left_index=True).reset_index()

In [ ]:
values= prediction['cluster_num'].unique()
print(values)
#clustering=agglomerative_clusters ##estos son los valores que arroja el algoritmo 
#values= np.unique(agglomerative_clusters)


#lo unico importante aqui es la ultima linea: df final debe tener todos los atributos, incluyendo la etiquieta de cluster 
data_agl = (
    paises_covid_df
        .set_index('Country/Region')
        .merge(prediction, right_index=True, left_index=True)
        .reset_index()
        .copy()
    )
data_agl['cluster_num'] = prediction ####nombrar de igual forma la columna por fis <<cluster_num>>
# data_agl = data_agl[['region', 'pais', 'cluster_num']]
# data_agl_comp = pd.merge(data_agl, dset_extra, how='inner')
# data_agl_comp = dset_base.merge(data_agl_comp.set_index('pais'), on='pais')
# nombrar el dataset como data_cluster
data_cluster=data_agl
clustering='tipo de cluster' ##ver ejemplo (tomado del hito 1)

interes=[ 'gdp_rank', 'gdp_usd', 'gdp_en_salud', 'edad_media', 'gini', 'poblacion', 'taza_muerte1000' ]
colores= ['darkorange', 'springgreen', 'lightseagreen', 'orchid', 'chocolate', 'lightsteelblue', 'indianred', 'cadetblue', 'gold']

datas={}
for i in range(0, len(values)): #separamos y subimos al diccionario
    llave=values[i]
    datai = data_cluster.loc[data_cluster['cluster_num']==values[i]]
    datas[llave]=datai

    
#estadisticas generales
"""
for i in range(0,len(values)):
    print('info gnral cluster ', values[i], ':')
    for a in range(0,len(interes)):
        datos=datas[values[i]]
        print(interes[a], 'promedio: ', datos[interes[a]].mean(), ';  desviacion:   ', datos[interes[a]].std() )
        print('---------------------------------------')
    print('*******************', 'fin cluster <', values[i] ,'>************************')
"""


#Paises en cluster
for i in range(0,len(values)):
    print('cluster', values[i])
    data=datas[values[i]]
    column_valxs = data[['Country/Region']].values
    unique_values =  np.unique(column_valxs)    
    print(unique_values)
    print('-----------------------------------------------------------------------------')


#Boxplots
for col in interes:
    par=col
    data_cluster.groupby('cluster_num')
    f, ax = plt.subplots(figsize=(7,5))
    img=sns.boxplot(y=par, x= 'cluster_num', data=data_cluster, width=0.3, showfliers = True) #showf False no muestra outliers


#densidades
for col in interes:
    par=col
    plt.figure()
    plt.title(col)
    for i in range(0,len(datas)):
        datos=datas[i]
        datos=list(datos[col])
        sns.distplot(datos, hist=False, color=colores[i])
    plt.close


#matrices corr atributos
for i in range(0,len(values)):
    print('para cluster:  ', values[i])
    data=datas[values[i]]
    # Incluir en la metodología
    mm_scaler = preprocessing.MinMaxScaler()
    data = data.drop(["fecha_observacion"  ,"region_x", "region_y","cluster_num", 'Country/Region', "num_dia_desde_22_enero", "num_dia_desde_primer_caso" ],axis=1)
    scaled_at = mm_scaler.fit_transform(data)
    scaled = pd.DataFrame(scaled_at)
    cormatrix = scaled.corr()
    f, ax = plt.subplots()
    sns.heatmap(cormatrix,vmax=1,vmin=-1,center=0, cmap='gnuplot2' )


#Algunas curvas(por cluster)
for i in values:
    print('llave: ', i)
    dato=datas[i]
    column_valxs = dato[['Country/Region']].values
    muestra = random.choices(np.unique(column_valxs), k=5)
    k=0
    var='num_dia_desde_primer_caso' #se puede hacer otro for o copiar el codigo cambiando esto para muertos y etc
    plt.figure()
    plt.title('cluster'+ str(i))
    for k in range(0,len(muestra)):
        pais=muestra[k]
        print('pais: ', pais)
        datos=data_cluster.loc[data_cluster['Country/Region']==pais]
        sns.lineplot(x=var, y='acc_confirmados', data=datos, color=colores[k])
    plt.close  

In [ ]:
data_cluster